In [22]:
import os
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from scipy.spatial.distance import cdist

In [23]:
def dataLoader(root, resize_factor=0):
    img_data = []
    label = []
    for file in sorted(os.listdir(root)):
        img_path = os.path.join(root, file)
        img = Image.open(img_path)

        w, h = img.size
        if resize_factor > 0: # Resize
            w, h = w//resize_factor, h//resize_factor
            img = img.resize((w, h))

        img = np.array(img).reshape(-1) # Reshape to 1*n, n=w*h
        img_data.append(img)
        label.append(int(file[7:9]))
    return np.array(img_data, dtype=np.float64), np.array(label), w, h

In [24]:
def normalize_img(img):
    # Normalize to [0, 255]
    normalize_img = ((img - np.min(img)) / (np.max(img) - np.min(img)))
    return (normalize_img * 255).astype(np.uint8)

def plot_eigenface(W, width, height, opt, cols=5):
    num_feature = opt["num_feature"]

    fig = plt.figure(figsize=(10, 10))
    for i in range(num_feature):
        img = W[:, i].reshape(height, width) # Eigenfaces and fisherfaces
        img = normalize_img(img)

        ax = fig.add_subplot(int(np.ceil(num_feature / cols)), cols, i+1)
        ax.axis('off')
        ax.imshow(img, cmap='gray')
        
    plt.tight_layout()
    fig.savefig(os.path.join(opt["save_root"], f'{opt["task"]}_eigenface.jpg'))
    plt.show()

def reconstruction(W, X, width, height, opt, cols=5):
    data_size = len(X)
    num_reconstruction = opt["num_reconstruction"]

    # Randomly pick some images to show their reconstruction
    idx = np.random.choice(data_size, num_reconstruction, replace=False)
    
    fig = plt.figure(figsize=(10, 3))
    for i in range(len(idx)):
        data = X[idx[i]]
        data_reshaped = data.reshape(height, width)

        # Reconstruction x * W * W^T
        reconstructed_img = (data @ W @ W.T).reshape(height, width)
        reconstructed_img = normalize_img(reconstructed_img)
        img = np.hstack([data_reshaped, reconstructed_img])

        ax = fig.add_subplot(int(np.ceil(len(idx) / cols)), cols, i+1)
        ax.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.imshow(img, cmap='gray')

    plt.tight_layout()
    fig.savefig(os.path.join(opt["save_root"], f'{opt["task"]}_reconstructed.jpg'))
    plt.show()

In [25]:
def pca(train, opt, kernel=False):
    if kernel:
        matrix = train
    else:
        # Compute covariance matrix
        matrix = np.cov(train.T)

    # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(matrix)

    # Find k eigenvectors corresponding to k largest eigenvalues
    sorted_indices = np.argsort(eigenvalues)[::-1]
    top_k_indices = sorted_indices[:opt["num_feature"]]
    W = eigenvectors[:, top_k_indices]
    
    # Normalized
    W = W / np.linalg.norm(W, axis=0, keepdims=True)
    return W

def lda(train, label, opt):
    data_size = train.shape[1]

    between_class = np.zeros((data_size, data_size), dtype=np.float64)
    within_class = np.zeros((data_size, data_size), dtype=np.float64)
    mean = np.mean(train, axis = 0) # Mean of all data

    for c in np.unique(label):
        X_c = train[label == c] # Data belonging to class c
        mean_c = np.mean(X_c, axis=0) # Class mean

        # Sum{(X_c - m_c)^t*(X_c - m_c)}
        within_class += (X_c - mean_c).T @ (X_c - mean_c)
        # Sum{n_c*(m_c - m)*(m_c - m)^t} 
        diff = (mean_c - mean).reshape(-1, 1) 
        between_class += len(X_c) * (diff @ diff.T)

    # Get first K largest eigenvectors of S_w^-1 * S_B
    S = np.linalg.pinv(within_class) @ between_class
    
    eigenvalues, eigenvectors = np.linalg.eigh(S)
    sorted_indices = np.argsort(eigenvalues)[::-1]
    top_k_indices = sorted_indices[:opt["num_feature"]]
    W = eigenvectors[:, top_k_indices]

    # Normalized
    W = W / np.linalg.norm(W, axis=0, keepdims=True)
    return W

def recognition(train, labelTr, test, labelTs, W, opt):
    X_W = train @ W
    Y_W = test @ W

    # K-NN
    error = 0
    K = opt["K-NN_k"]
    for i in range(len(test)):
        dist = np.zeros(len(train))
        for j in range(len(train)):
            # Distance between training data and testing data
            dist[j] = np.sum((X_W[j] - Y_W[i]) ** 2)

        nearest_neighbor = np.argsort(dist)[:K] # Find the K nearest neighbor
        k_nearest_labels = [labelTr[n] for n in nearest_neighbor]
        label_count = {} # Assign label
        for label in k_nearest_labels:
            if label in label_count:
                label_count[label] += 1
            else:
                label_count[label] = 1
        predict = max(label_count, key=label_count.get)

        if predict != labelTs[i]:
            error += 1
    accuracy_rate = 1 - (error / len(test))
    print(f'Task: {opt["task"]}, ACC: {accuracy_rate:.4f}, Error: {error}') 

In [26]:
def center_kernel(K):
    row_mean = np.mean(K, axis=1, keepdims=True)
    col_mean = np.mean(K, axis=0, keepdims=True)
    overall_mean = np.mean(K)
    K_centered = K - row_mean - col_mean + overall_mean
    return K_centered

def build_kernel(u, v, opt, gamma=1e-8, center=True):
    if opt["kernel_type"] == 'RBF':
        sq_dist = np.sum(u**2, axis=1)[:, None] + np.sum(v**2, axis=1)[None, :] - 2 * (u @ v.T)
        RBF = np.exp(-gamma * sq_dist)
        if center:
            RBF = center_kernel(RBF)
        return RBF
    elif opt["kernel_type"] == 'Linear':
        linear = u @ v.T
        if center:
            linear = center_kernel(linear)
        return linear
    elif opt["kernel_type"] == 'LinearRBF':
        linear = u @ v.T
        sq_dist = np.sum(u**2, axis=1)[:, None] + np.sum(v**2, axis=1)[None, :] - 2 * (u @ v.T)
        RBF = np.exp(-gamma * sq_dist)
        linearRBF = linear + RBF
        if center:
            linearRBF = center_kernel(linearRBF)
        return linearRBF

In [27]:
def main(opt):
    imageTr, labelTr, w_Tr, h_Tr = dataLoader(opt["dataTr_path"], resize_factor=opt["resize"])
    imageTs, labelTs, _, _ = dataLoader(opt["dataTs_path"], resize_factor=opt["resize"])

    if opt["task"] == "PCA":
        W = pca(imageTr, opt)
        plot_eigenface(W, w_Tr, h_Tr, opt)
        reconstruction(W, imageTr, w_Tr, h_Tr, opt)
        recognition(imageTr, labelTr, imageTs, labelTs, W, opt)

    elif opt["task"] == "LDA":
        W = lda(imageTr, labelTr, opt)
        plot_eigenface(W, w_Tr, h_Tr, opt)
        reconstruction(W, imageTr, w_Tr, h_Tr, opt)
        recognition(imageTr, labelTr, imageTs, labelTs, W, opt)

    elif opt["task"] == "kernelPCA":
        kernelTr = build_kernel(imageTr, imageTr, opt)
        kernelTs = build_kernel(imageTs, imageTr, opt)
        W = pca(kernelTr, opt, kernel=True)
        recognition(kernelTr, labelTr, kernelTs, labelTs, W, opt)

    elif opt["task"] == "kernelLDA":
        kernelTr = build_kernel(imageTr, imageTr, opt)
        kernelTs = build_kernel(imageTs, imageTr, opt)
        W = lda(kernelTr, labelTr, opt)
        recognition(kernelTr, labelTr, kernelTs, labelTs, W, opt)

In [ ]:
if __name__ == "__main__":
    opt = {
        "dataTr_path"        : "Yale_Face_Database/Training",
        "dataTs_path"        : "Yale_Face_Database/Testing",
        "save_root"          : "results_new",
        "task"               : "LDA", # PCA, LDA, kernelPCA, kernelLDA
        "kernel_type"        : "RBF", # RBF, Linear, LinearRBF
        "num_feature"        : 25,
        "num_reconstruction" : 10,
        "K-NN_k"             : 5,
        "resize"             : 5
    }
    os.makedirs(opt["save_root"], exist_ok=True)
    main(opt)